In [1]:
import torch
import transformers

In [2]:
from gpt4all import GPT4All
model = GPT4All("orca-mini-3b-gguf2-q4_0.gguf")

In [3]:
def stop_on_token_callback(token_id, token_string):
    if '.' in token_string: return False
    else: return True
response = model.generate("Summarize this news, BlackRock’s spot bitcoin exchange-traded fund (ETF), which trades under the ticker IBIT on Nasdaq, fell out of favor on Wednesday, preliminary data published by Farside Investors showed. For the first time since going live on January 11, the fund did not draw any investor money, snapping a 71-day inflows streak. Seven out of the other ten funds followed IBIT’s lead. Fidelity’s FBTC and the ARK 21Shares Bitcoin ETF (ARKB) registered inflows of $5.6 million and $4,2 million, respectively, while Grayscale’s GBTC bled $130.4 million, leading to a new cumulative outflow of $120.6 million, the highest since April 17. Spot ETFs debuted in the U.S. on January 11 with much fanfare, promising to pull billions of dollars in institutional money. To date, BlackRock’s IBIT has alone amassed more than $15 billion, while the 11 funds taken together have registered a net inflow of over $12 billion. However, most inflows happened in the first quarter, and the uptake has slowed this month, taking the wind out of the bitcoin bull run.Bitcoin (BTC), the leading cryptocurrency by market value, has traded primarily between $60,000 and $70,000 this month, CoinDesk data show, marking a weak follow-through to the first quarter’s near 70% rally to record highs above $73,500.", temp= 0, callback=stop_on_token_callback)
print(response)

 The current pullback could be an opportunity for investors to add some exposure to the market at lower prices, according to analysts.


In [45]:
new_api = "6064ea25c53d48a3abd8159d80cc92c6"
newsapi_url = "https://newsapi.org/v2/everything?q=cryptocurrency&apiKey=6064ea25c53d48a3abd8159d80cc92c6"


In [59]:
import requests
response = requests.get(newsapi_url)

if response.status_code == 200:
    print("Status ok")
else:
    print("Error:", response.status_code)

Status ok


In [100]:
json_data = response.json()
json_data.keys()

dict_keys(['status', 'totalResults', 'articles'])

In [96]:
json_data["articles"][1].keys()

dict_keys(['source', 'author', 'title', 'description', 'url', 'urlToImage', 'publishedAt', 'content'])

In [88]:
import json
from bs4 import BeautifulSoup
import numpy as np
from datetime import datetime
import datefinder


from transformers import pipeline
pipe = pipeline("text-classification", model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
def analyize_sentimental_text(text):
    return pipe(text)

def split_text(text, max_length=514):
    sentences = []
    current_sentence = ""
    for word in text.split():
        if len(current_sentence) + len(word) < max_length:
            current_sentence += " " + word if current_sentence else word
        else:
            if not current_sentence.endswith("."):
                current_sentence += "."
            sentences.append(current_sentence)
            current_sentence = word
    if current_sentence:
        if not current_sentence.endswith(".") & len(current_sentence) < max_length:
            current_sentence += "."
        sentences.append(current_sentence)
    return sentences

def append_json_data(path, new_data):
    # update json file
    with open(path, 'r') as extracted_text_data:
        original_data = json.load(extracted_text_data)
    original_data.append(new_data)
    with open(path, 'w') as extracted_text_data:
        json.dump(original_data, extracted_text_data, indent=4)

In [89]:
def avg_score_finder(sentimental_scores):
    labels = {'neutral' : 0, 'negative': -1, 'positive': 1}
    scores: list = [labels[sentimental_score[0]["label"]] for sentimental_score in sentimental_scores]
    return np.average(scores)

This following code is to extract the most common Currency that is mentioned in the text

In [90]:
import tensorflow as tf
import yfinance as yf
import pandas as pd
from collections import Counter

In [91]:
crypto_dataset = pd.read_csv("data/archive/Cryptocurrency_Dataset_2021.csv")
crypto_dataset

,Symbol,Name,Price (Intraday),Change,% Change,Market Cap,Volume in Currency (Since 0:00 UTC),Volume in Currency (24Hr),Total Volume All Currencies (24Hr),Circulating Supply
0,BTC-USD,Bitcoin USD,"51,013.81","1,741.86",0.0374,963.866B,34.642B,34.642B,34.642B,18.894M
1,ETH-USD,Ethereum USD,"4,310.12",123.49,0.0295,511.373B,23.695B,23.695B,23.695B,118.645M
2,BNB-USD,BinanceCoin USD,579.82,10.85,0.0191,96.714B,2.376B,2.376B,2.376B,166.801M
3,USDT-USD,Tether USD,0.999563,-0.000893,-0.09%,75.981B,79.634B,79.634B,79.634B,76.014B
4,SOL1-USD,Solana USD,194.93,5.62,0.0297,59.631B,2.804B,2.804B,2.804B,305.911M
...,...,...,...,...,...,...,...,...,...,...
390,GRN-USD,GreenPower USD,0.012171,-0.001566,-11.40%,0,50,50,50,0
391,KNC-USD,KyberNetwork USD,1.5066,0.152,0.1122,0,"128,861","128,861","128,861",0
392,BONFIRE-USD,Bonfire USD,0.00000002,0,0.1971,0,"124,365","124,365","124,365",0
393,BST-USD,BlockStamp USD,0.066326,0.002342,0.0366,0,0,0,0,0


In [92]:
currencies = [name for name in crypto_dataset["Name"]]
entities: list = []
for currency in currencies:
    entities.append(currency.split()[0])

def get_entity(prompt: str):
    matching_words = [word for word in entities if word in prompt]
    # most_common_cur_mention, frequency = Counter(matching_words).most_common(1)[0]
    most_common_cur_mention = Counter(matching_words).most_common(4)
    return most_common_cur_mention

Extract the data from the web & News API

In [101]:
for article in json_data["articles"]:
    source, author, title, description, url, urlToImage, published_at, content = [article[keys] for keys in article.keys()]
    web_response = requests.get(url)
    if web_response.status_code == 200:
            soup = BeautifulSoup(web_response.text, 'html.parser')
            paragraphs = soup.find_all('p')
            paragraph_texts = [paragraph.getText(strip=True) for paragraph in paragraphs]
            combined_paragraph_text = ' '.join(paragraph_texts)

            limit_paragraph = split_text(combined_paragraph_text)
            currency_mention = get_entity(title + combined_paragraph_text)
            title_sentimental = [analyize_sentimental_text(title)]
            overall_sentimental_scores = [analyize_sentimental_text(paragraph_sentence) for paragraph_sentence in limit_paragraph]
            sentimental_scores = [analyize_sentimental_text(paragraph_text) for paragraph_text in paragraph_texts]

            avg_title_sentimental = avg_score_finder(title_sentimental)
            avg_sentimental_sentimental = avg_score_finder(sentimental_scores)
            avg_overall_sentimental = avg_score_finder(overall_sentimental_scores)


            print(f'{title}\n'
                  f'title sentimental score {avg_title_sentimental} '
                  f'sentimental avg. score {avg_sentimental_sentimental}  '
                  f'overall sentimental_score {avg_overall_sentimental}')

            update_into_payload = {
                "Currency" : currency_mention if currency_mention else None,
                "paragraph" : paragraph_texts,
                "avg title sentimental": avg_title_sentimental,
                "avg sentimental sentimental" : avg_sentimental_sentimental,
                "avg overall sentimental" : avg_overall_sentimental,
                "headline sentimental" : title_sentimental,
                "sentimental" : sentimental_scores,
                "overall sentimental score" : overall_sentimental_scores
            ,}
            article.update(update_into_payload)

            # update json extracted_text_data.json
            append_json_data("data/extracted_text_data.json", article)
    else: print("Failed to fetch the webpage. Status code:", web_response.status_code)

No one needs this cryptocurrency-powered Steam Deck competitor
title 0.0 avg. 0.047619047619047616  overall0.0
Most Trending Cryptocurrency Today – Ethena, Meme Ai, Dogwifhat
title 0.0 avg. 0.125  overall0.3125
Venezuela bets on Tether cryptocurrency to skirt oil sanctions
title 0.0 avg. 0.11538461538461539  overall0.0
Crypto influencer guilty of $110M scheme that shut down Mango Markets
title -1.0 avg. -0.05555555555555555  overall-0.125
Drug boss ordered to hand over £3.5m
title 0.0 avg. -0.07317073170731707  overall-0.14285714285714285
A London millennial who turned about $6,000 into $30,000 through cryptocurrency investing shares his top investment tips
title 0.0 avg. 0.125  overall0.3
Change Healthcare Finally Admits It Paid Ransomware Hackers—and Still Faces a Patient Data Leak
title 0.0 avg. -0.14285714285714285  overall-0.17647058823529413
Goldman Sachs executive remains skeptical about Bitcoin
title -1.0 avg. 0.1  overall0.3333333333333333
Bitcoin, altcoins dip on Iran-Israel 


This will be part of the main Training Algorithm